In [1]:
import random
import math
from itertools import permutations
import numpy as np
from scipy.spatial.transform import Rotation as R

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

Using TensorFlow backend.


### Generating Model
Define the generating model, that will be used to create the training data

In [4]:
all_rotation_orders = [''.join(axes) for axes in permutations({'x','y','z'})]
print(all_rotation_orders)

['zyx', 'zxy', 'yzx', 'yxz', 'xzy', 'xyz']


In [9]:
def random_vector(n, mag):
    """
    helper function to generate a vector in random direction of given magnitude
    """
    return 2.0 * mag * (0.5-np.random.random(size=n))

# try it out
print(random_vector(10,5.0))

[ 4.34739145 -2.02036859  3.63143152  0.88579205  2.52045845 -0.92837115
 -1.43048254  4.15623379  4.11865025  4.76572516]


In [10]:
def generate_sample_from_model():
    """
    a function to generate a sample from the model.  sample is a tuple of:
    * np.array with 9 matrix elements + 3 rotation angles
    * string corresponding to chosen rotation order (element of all_rotation_orders)    
    """
    rotation_order = random.choice(all_rotation_orders)  # choose a rotation order
    angles = random_vector(3, 10.0)   # generate three random angles (in degrees)
    if (min(abs(angles)) < 0.1):     # reject any angles that are too small
        return generate_sample_from_model()
    matrix = R.from_euler(rotation_order, angles, degrees = True)  # generate matrix
    noise_angles = angles + random_vector(3, 0.1)    
    x = np.concatenate((matrix.as_dcm().flatten(), 
                        np.array(noise_angles)))
                        # np.array(np.cos(noise_angles)), 
                        # np.array(np.sin(noise_angles))))                         
    return (x, rotation_order)

# try it out
print(generate_sample_from_model())

(array([ 0.99634269,  0.06658766, -0.05354749, -0.06062162,  0.99249287,
        0.10622107,  0.06021851, -0.10258645,  0.99289967, -5.91173804,
       -3.43051804, -3.3871399 ]), 'xyz')


In [11]:
def generate_samples(n, mean=None, std=None):
    """
    define a function to generate multiple samples, suitable for training or testing.
    for number N, sample is a tuple:
    * np.array Nx12 of x vectors (standardized zero mean)
    * np.array Nx6 one-hot vectors
    """
    [xs, rotation_orders] = list(zip(*[generate_sample_from_model() for _ in range(n)]))
    xs = np.array(xs)
    if mean is None:
        mean = xs.mean(0)
    if std is None:
        std = xs.std(0)
    xs -= mean    # standardize along sample dimension
    xs /= std
    ys = [all_rotation_orders.index(rotation_order) for rotation_order in rotation_orders]
    return xs, mean, std, keras.utils.to_categorical(ys)
    
# try it out
print(generate_samples(3))

(array([[ 0.12189008, -0.72978367, -0.23702413,  0.78182226, -0.22942904,
        -0.59296971,  0.23571431,  0.62678168,  0.47989768,  0.72324012,
         1.41113141, -1.26569055],
       [ 1.15924228,  1.4139685 ,  1.32593273, -1.41148228,  1.323235  ,
         1.40837073, -1.32547012, -1.41127906,  0.91212479, -1.41408992,
        -0.62475037,  0.0864749 ],
       [-1.28113237, -0.68418482, -1.0889086 ,  0.62966002, -1.09380595,
        -0.81540102,  1.08975581,  0.78449738, -1.39202248,  0.6908498 ,
        -0.78638103,  1.17921564]]), array([ 0.98300445, -0.06187748, -0.0526789 ,  0.04719734,  0.97815231,
       -0.06291833,  0.07438123,  0.04833613,  0.98367378,  3.42379269,
       -3.30476701,  1.98056427]), array([4.88595628e-03, 1.41449885e-01, 8.40620334e-02, 1.35971367e-01,
       2.18824195e-03, 1.36157117e-01, 8.61975398e-02, 1.30537364e-01,
       6.84964817e-03, 7.65894156e+00, 8.17713443e+00, 4.93218561e+00]), array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.,

In [12]:
# generate testing and training data
x_train, mean, std, y_train = generate_samples(10000)
x_test, _, _, y_test = generate_samples(1000, mean, std)
print(x_train.shape, y_train.shape)

(10000, 12) (10000, 6)


### Network Model
Now create the MLP model that will be trained on the samples

In [13]:
# now create dense MLP model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim = x_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 198       
Total params: 4,166
Trainable params: 4,166
Non-trainable params: 0
_________________________________________________________________


In [14]:
# create the optimizer and compile the network model
adam = Adam(lr=0.001, decay=1e-8)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [15]:
# now fit the data using training samples
model.fit(x_train, y_train, epochs=200, batch_size=256)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
10000/10000 [==============================] - 4s 420us/step - loss: 1.7257 - acc: 0.2784
Epoch 2/200
10000/10000 [==============================] - 0s 13us/step - loss: 1.4705 - acc: 0.4433
Epoch 3/200
10000/10000 [==============================] - 0s 13us/step - loss: 1.0786 - acc: 0.6151
Epoch 4/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.7428 - acc: 0.7451
Epoch 5/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.5431 - acc: 0.8337
Epoch 6/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.4353 - acc: 0.8682
Epoch 7/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.3644 - acc: 0.8903
Epoch 8/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.3177 - acc: 0.9072
Epoch 9/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.2828 - acc: 0.9164
Epoch 10/200
10000/10000 [================

10000/10000 [==============================] - 0s 13us/step - loss: 0.0485 - acc: 0.9862
Epoch 79/200
10000/10000 [==============================] - 0s 13us/step - loss: 0.0494 - acc: 0.9851
Epoch 80/200
10000/10000 [==============================] - 0s 13us/step - loss: 0.0505 - acc: 0.9862
Epoch 81/200
10000/10000 [==============================] - 0s 16us/step - loss: 0.0512 - acc: 0.9850
Epoch 82/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0500 - acc: 0.9851
Epoch 83/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0491 - acc: 0.9852
Epoch 84/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0794 - acc: 0.9726
Epoch 85/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0554 - acc: 0.9822
Epoch 86/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0534 - acc: 0.9814
Epoch 87/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0477 - acc: 

10000/10000 [==============================] - 0s 15us/step - loss: 0.0257 - acc: 0.9929
Epoch 158/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0277 - acc: 0.9917
Epoch 159/200
10000/10000 [==============================] - 0s 13us/step - loss: 0.0253 - acc: 0.9935
Epoch 160/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0267 - acc: 0.9923
Epoch 161/200
10000/10000 [==============================] - 0s 13us/step - loss: 0.0346 - acc: 0.9879
Epoch 162/200
10000/10000 [==============================] - 0s 13us/step - loss: 0.0246 - acc: 0.9932
Epoch 163/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0249 - acc: 0.9931
Epoch 164/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0258 - acc: 0.9923
Epoch 165/200
10000/10000 [==============================] - 0s 14us/step - loss: 0.0586 - acc: 0.9801
Epoch 166/200
10000/10000 [==============================] - 0s 13us/step - loss: 0.028

In [16]:
# try out the model
score = model.evaluate(x_test, y_test, batch_size=256)
print(score)

1000/1000 [==============================] - 0s 54us/step
[0.04179360863566399, 0.9840000047683716]


In [18]:
model.save('sro_decoder_mlp_model.h5')